In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import warnings                                      
warnings.filterwarnings('ignore')

import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns


In [2]:
df = pd.read_csv("input.csv")
df.head()

df_test = pd.read_csv("test.csv")

In [3]:
df = df.loc[df['콘존ID'].str.contains('0010', na = False)]
#df_test = df_test.loc[df_test['콘존ID'].str.contains('0010', na = False)]

In [4]:
#정체 정도에 따라 라벨링
congestion = []
#def labeling
for row in df['평균속도']:
    if row <= 40:
        congestion.append(2)
    elif row <= 80:
        congestion.append(1)
    else:
        congestion.append(0)

df['congestion'] = congestion
df.head()

#df_test['congestion'] = congestion

,Unnamed: 0,수집시분초,평균속도,교통량,콘존ID,차로번호,congestion
0,0,0,90.0,3.0,0010CZE010,1,0
1,1,0,82.0,3.0,0010CZE010,2,0
2,2,0,58.0,1.0,0010CZE010,3,1
3,6,0,104.0,1.0,0010CZE011,1,0
4,8,0,61.0,1.0,0010CZE011,3,1


In [5]:
df = df.loc[:, ['수집시분초', '평균속도', '교통량', '콘존ID', '차로번호', 'congestion']]
df

,수집시분초,평균속도,교통량,콘존ID,차로번호,congestion
0,0,90.0,3.0,0010CZE010,1,0
1,0,82.0,3.0,0010CZE010,2,0
2,0,58.0,1.0,0010CZE010,3,1
3,0,104.0,1.0,0010CZE011,1,0
4,0,61.0,1.0,0010CZE011,3,1
...,...,...,...,...,...,...
103408193,235930,70.0,4.0,0010CZS280,2,1
103408194,235930,62.0,2.0,0010CZS280,3,1
103408195,235930,91.0,5.0,0010CZS380,2,0
103408196,235930,21.0,2.0,0010CZS380,3,2


In [6]:
#00초만 남기고 제거
idx_sec = df[(df['수집시분초'] % 100) != 0].index

#15분 간격만 남기고 제거
df = df.drop(idx_sec)
#idx_min = df[((df['수집시분초'] % 10000) != 0) &
#            ((df['수집시분초'] % 10000) != 1500) &
#            ((df['수집시분초'] % 10000) != 3000) &
#            ((df['수집시분초'] % 10000) != 4500)].index

#5분 간격만 남기고 제거
idx_min = df[(df['수집시분초'] / 50) == 0].index

df = df.drop(idx_min)
df

,수집시분초,평균속도,교통량,콘존ID,차로번호,congestion
4291,10400,77.0,3.0,0010CZE010,2,1
4364,10400,90.0,1.0,0010CZE011,2,0
4365,10400,90.0,1.0,0010CZE011,2,0
4366,10400,72.0,1.0,0010CZE030,2,1
4367,10400,72.0,1.0,0010CZE035,3,1
...,...,...,...,...,...,...
103407483,235900,56.0,3.0,0010CZS380,1,1
103407484,235900,83.0,3.0,0010CZS380,2,0
103407485,235900,95.0,2.0,0010CZS380,3,0
103407486,235900,62.0,2.0,0010CZS380,4,1


In [7]:
#Encoding(label, one-hot, etc.)
encoding_columns = ['수집시분초', '차로번호']
not_encoding_columns = ['교통량', '평균속도', '콘존ID', 'congestion']

enc_classes = {}
def encoding_label(x):
    le = LabelEncoder()
    le.fit(x)
    label = le.transform(x)
    
    enc_classes[x.name] = le.classes_
    
    return label

d1 = df[encoding_columns].apply(encoding_label)
d2 = df[not_encoding_columns]

#traffic congestion
data = d1.join(d2)
#data

data.describe()

,수집시분초,차로번호,교통량,평균속도,congestion
count,9.739259e+06,9.739259e+06,9.739259e+06,9.739259e+06,9.739259e+06
mean,7.647951e+02,1.403649e+00,4.733643e+00,9.363555e+01,2.334089e-01
std,3.859670e+02,1.106427e+00,3.573906e+00,1.953184e+01,4.553965e-01
min,0.000000e+00,0.000000e+00,1.000000e+00,5.000000e+00,0.000000e+00
25%,4.590000e+02,1.000000e+00,2.000000e+00,8.200000e+01,0.000000e+00
50%,7.740000e+02,1.000000e+00,4.000000e+00,9.300000e+01,0.000000e+00
75%,1.090000e+03,2.000000e+00,7.000000e+00,1.050000e+02,0.000000e+00
max,1.438000e+03,6.000000e+00,3.000000e+01,1.500000e+02,2.000000e+00


In [ ]:
#sns.pairplot(data, hue = 'congestion')

In [ ]:
X = pd.get_dummies(data, columns = ['콘존ID'])

df_temp = X.copy()
df_temp.head()

In [ ]:
from sklearn.model_selection import train_test_split

y = df_temp['congestion']
X = df_temp.drop(columns = 'congestion')

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 2)
print(X.shape, X_train.shape, X_test.shape)

In [ ]:
#GridSearchCV를 이용한 하이퍼 파라미터 튜닝
'''
params = {'max_depth' : [6, 8, 10, 12],
          'min_samples_leaf' : [8, 12, 18],
          'min_samples_split' : [8, 16, 20]
         }
classifier = RandomForestClassifier(random_state = 0, n_jobs = -1)
grid_cv = GridSearchCV(classifier, param_grid = params, cv = 3, n_jobs = -1)
grid_cv.fit(X_train, y_train)

print("최적 하이퍼 파라미터: ", grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))
'''

In [ ]:
# n_estimator: 결정트리의 갯수를 지정, 많을수록 좋으나 학습시간 증가

#min_samples_spilt: 노드를 분할하기 위한 최소한의 샘플 데이터수(과적합 제어에 사용)
#작게 설정할수록 분할 노드가 많아져 과적합 가능성 증가

#min_samples_leaf: 리프노드가 되기 위해 필요한 최소한의 샘플 데이터수
#min_samples_split과 함께 과적합 제어 용도
#불균형 데이터의 경우 특정 클래스의 데이터가 극도로 작을 수 있으므로 작게 설정 필요

#max_features: 최적의 분할을 위해 고려할 최대 feature 개수
#int형으로 지정 -> 피처 개수 / float형으로 지정 -> 비중
#sqrt 또는 auto: 전체 피처 중 root(피처개수)만큼 선정
#log: 전체 피처 중 log2(전체 피처개수)만큼 선정

#max_depth: 트리의 최대 깊이, 완벽하게 클래스 값이 결정될 때 까지 분할
#또는 데이터 개수가 min_samples_split보다 작아질 때까지 분할
#깊이가 깊어지면 과적합 될 수 있으므로 적절히 제어 필요

#max_leaf_nodes
#리프 노드의 최대 개수

classifier = RandomForestClassifier(
    max_depth = 6,
    min_samples_leaf = 6,
    min_samples_split = 12,
    n_estimators = 100
)

classifier.fit(X_train, y_train)

pred_train = classifier.predict(X_train)
pred_test = classifier.predict(X_test)
acc_train = accuracy_score(y_train, pred_train)
acc_test = accuracy_score(y_test, pred_test)

print(f'학습: {acc_train}, 테스트: {acc_test}')

In [ ]:
#sns.scatterplot(x = X[:, '평균속도'], y = X[:, '교통량'], hue = 'congestion', data = df)


In [ ]:
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

def print_score(classifier, X_train, y_train, X_test, y_test, train = True):
    if train == True:
        print("Training results:\n")
        print("Accuracy Score: {0:.4f}\n".format(accuracy_score(y_train, classifier.predict(X_train))))
        print("Classification Report:\n{}\n".format(classification_report(y_train, classifier.predict(X_train))))
        print("Confusion Matrix:\n{}\n".format(confusion_matrix(y_train, classifier.predict(X_train))))
        
        #res = cross_val_score(classifier, X_train, y_train, cv = 10, n_jobs = -1, scoring = 'accuracy')
        
        #print("Average Accuracy:\t{0:.4f}\n".format(res.mean()))
        #print("Standard Deviation:\t{0:.4f}\n".format(res.std()))
        
    elif train == False:
        print("Test results:\n")
        print("Accuracy Score: {0:.4f}\n".format(accuracy_score(y_test, classifier.predict(X_test))))
        print("Classification Report:\n{}\n".format(classification_report(y_test, classifier.predict(X_test))))
        print("Confusion Matrix:\n{}\n".format(confusion_matrix(y_test, classifier.predict(X_test))))

In [1]:
print_score(classifier, X_train, y_train, X_test, y_test, train = True)

NameError: name 'print_score' is not defined

In [ ]:
print_score(classifier, X_train, y_train, X_test, y_test, train = False)